<a href="https://colab.research.google.com/github/ychoi-kr/llm-api-prog/blob/main/5_gemini/live_text_to_voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-genai

In [ ]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google import genai
MODEL = "gemini-2.0-flash-exp"
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [ ]:
client = genai.Client(
    api_key=os.getenv("GEMINI_API_KEY"),
    http_options={"api_version": "v1alpha"}
)

config = {
    "generation_config": {
        "response_modalities": ["AUDIO"],
    },
    "system_instruction": "당신은 한국어로 대화하는 AI 어시스턴트입니다.",
}


In [ ]:
import asyncio
import wave
import io
from IPython.display import Audio, display

async def read_text_stream(text: str):
    words = text.split()
    for word in words:
        yield word
        await asyncio.sleep(0.1)  # 단어 간 약간의 지연

def play_audio(data: bytes, RATE=24000, CHANNELS=1):
    """오디오 재생"""
    # WAV 형식으로 변환
    wav_buffer = io.BytesIO()
    with wave.open(wav_buffer, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(2)  # 16-bit audio
        wf.setframerate(RATE)
        wf.writeframes(data)
    # 재생 위젯 표시
    display(Audio(data=wav_buffer.getvalue(), autoplay=True))

In [ ]:
async def main():
    async with client.aio.live.connect(model=MODEL, config=config) as session:
        try:
            messages = ["인공지능에 대해 짧게 설명하세요.", "그럼 의식도 있나요?"]
            for message in messages:
                # 텍스트 스트림 전송
                last_chunk = None
                async for chunk in read_text_stream(message):
                    print(f"전송: {chunk}")
                    if last_chunk is not None:
                        await session.send(last_chunk, end_of_turn=False)
                    last_chunk = chunk

                if last_chunk is not None:
                    await session.send(last_chunk, end_of_turn=True)

                    print("\n응답 수신 대기 중...")
                    response_chunks = []
                    async for response in session.receive():
                        if response.data is not None:
                            response_chunks.append(response.data)

                    if response_chunks:
                        print("응답 재생 중...")
                        complete_audio = b"".join(response_chunks)
                        play_audio(complete_audio)
                        print("\n계속 대화하시려면 메시지를 입력하세요...")

        except KeyboardInterrupt:
            print("\n대화를 종료합니다.")
        except Exception as e:
            print(f"오류 발생: {str(e)}")
            import traceback
            print(traceback.format_exc())
            raise e

await main()

전송: 인공지능에
전송: 대해
전송: 짧게
전송: 설명하세요.

응답 수신 대기 중...
응답 재생 중...



계속 대화하시려면 메시지를 입력하세요...
전송: 그럼
전송: 의식도
전송: 있나요?

응답 수신 대기 중...
응답 재생 중...



계속 대화하시려면 메시지를 입력하세요...
